# Module Import Cell

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

## Data Import Cell

In [2]:
#Available population density by state and year meassured in persons per square mile
pop_df = pd.read_csv('data/Historical_Population_Density_States_only_RAND_US_1.csv')
#---------------------------------------------------------------------------------------
#Available firearm background check data
nics_df = pd.read_csv('data/nics-firearm-background-checks.csv')

In [7]:
nics_df.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2020-02,Alabama,29633.0,604.0,24590.0,12531.0,1225.0,1217,0.0,45.0,...,0.0,0.0,0.0,32.0,17.0,3.0,0.0,3.0,0.0,75336
1,2020-02,Alaska,139.0,10.0,2560.0,1839.0,311.0,203,0.0,0.0,...,0.0,0.0,0.0,4.0,9.0,0.0,0.0,0.0,0.0,5466
2,2020-02,Arizona,6138.0,741.0,16839.0,8006.0,1424.0,1392,0.0,20.0,...,0.0,0.0,0.0,9.0,7.0,2.0,1.0,0.0,0.0,37100
3,2020-02,Arkansas,2994.0,731.0,8112.0,5453.0,363.0,542,4.0,15.0,...,0.0,0.0,0.0,18.0,8.0,4.0,0.0,0.0,0.0,21445
4,2020-02,California,32002.0,0.0,35570.0,22645.0,4221.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95430


In [9]:
nics_df.columns

Index(['month', 'state', 'permit', 'permit_recheck', 'handgun', 'long_gun',
       'other', 'multiple', 'admin', 'prepawn_handgun', 'prepawn_long_gun',
       'prepawn_other', 'redemption_handgun', 'redemption_long_gun',
       'redemption_other', 'returned_handgun', 'returned_long_gun',
       'returned_other', 'rentals_handgun', 'rentals_long_gun',
       'private_sale_handgun', 'private_sale_long_gun', 'private_sale_other',
       'return_to_seller_handgun', 'return_to_seller_long_gun',
       'return_to_seller_other', 'totals'],
      dtype='object')